In [1]:
from rouge_score import rouge_scorer
import polars as pl

OUTPUT_PATH = "results/rag_results.jsonl"

In [2]:
df = pl.read_ndjson(OUTPUT_PATH)

df

item_index,prompt_variant,question,gold_answer,context,documents,system_prompt,model_answer
i64,str,str,str,str,list[struct[2]],str,str
0,"""baseline""","""What is the primary function o…","""The mitochondria produce ATP a…","""Shakespeare wrote tragedies su…","[{""Shakespeare wrote tragedies such as Macbeth and Romeo and Juliet."",""irrelevant""}, {""The Eiffel Tower was completed in 1889 and stands as a symbol of Paris, France."",""irrelevant""}, … {""Basketball was invented by James Naismith in 1891."",""irrelevant""}]","""You are an assistant for quest…","""Mitochondria are responsible f…"
1,"""baseline""","""Who proposed the theory of gen…","""Albert Einstein proposed the t…","""Photosynthesis converts light …","[{""Photosynthesis converts light into chemical energy in plants."",""irrelevant""}, {""The ancient Romans built aqueducts to transport water across great distances."",""irrelevant""}, … {""Python is a high-level programming language known for its readability."",""irrelevant""}]","""You are an assistant for quest…","""Albert Einstein proposed the t…"
2,"""baseline""","""What gas do plants absorb duri…","""Plants absorb carbon dioxide d…","""The human heart contains four …","[{""The human heart contains four chambers: two atria and two ventricles."",""irrelevant""}, {""Carbon dioxide, water, and sunlight are essential components for producing glucose in plants."",""relevant""}, … {""The Pacific Ocean is the largest ocean on Earth."",""irrelevant""}]","""You are an assistant for quest…","""During photosynthesis, plants …"
3,"""baseline""","""What is the capital of Japan?""","""The capital of Japan is Tokyo.""","""The Pythagorean theorem relate…","[{""The Pythagorean theorem relates the sides of a right triangle."",""irrelevant""}, {""Jupiter has over 70 known moons orbiting it."",""irrelevant""}, … {""Diamonds are composed of carbon atoms arranged in a crystal lattice."",""irrelevant""}]","""You are an assistant for quest…","""Tokyo is the capital city of J…"
4,"""baseline""","""Which element has the chemical…","""The element with the symbol 'F…","""The symbol Fe originates from …","[{""The symbol Fe originates from the Latin word 'ferrum', meaning iron."",""relevant""}, {""The Taj Mahal was built during the Mughal Empire."",""irrelevant""}, … {""Whales are the largest mammals on Earth."",""irrelevant""}]","""You are an assistant for quest…","""The chemical symbol 'Fe' stand…"
…,…,…,…,…,…,…,…
15,"""baseline""","""What is the fastest land anima…","""The fastest land animal is the…","""Diamonds are formed under high…","[{""Diamonds are formed under high pressure deep underground."",""irrelevant""}, {""Bats are the only mammals capable of sustained flight."",""irrelevant""}, … {""Chocolate is made from cacao seeds."",""irrelevant""}]","""You are an assistant for quest…","""The cheetah is the fastest lan…"
16,"""baseline""","""Which continent is the Sahara …","""The Sahara Desert is located i…","""The Pythagorean theorem relate…","[{""The Pythagorean theorem relates the sides of a right triangle."",""irrelevant""}, {""Apples grow on deciduous trees."",""irrelevant""}, … {""Butterflies have four wings."",""irrelevant""}]","""You are an assistant for quest…","""The Sahara Desert is located o…"
17,"""baseline""","""Who was the first president of…","""George Washington was the firs…","""George Washington served as th…","[{""George Washington served as the first president of the United States from 1789 to 1797."",""relevant""}, {""Basketball was invented in the 19th century."",""irrelevant""}, … {""Washington played a key role in the American Revolution and the founding of the nation."",""relevant""}]","""You are an assistant for quest…","""George Washington was the firs…"


In [3]:
df["gold_answer"]

gold_answer
str
"""The mitochondria produce ATP a…"
"""Albert Einstein proposed the t…"
"""Plants absorb carbon dioxide d…"
"""The capital of Japan is Tokyo."""
"""The element with the symbol 'F…"
…
"""The fastest land animal is the…"
"""The Sahara Desert is located i…"
"""George Washington was the firs…"


In [4]:
df["model_answer"]

model_answer
str
"""Mitochondria are responsible f…"
"""Albert Einstein proposed the t…"
"""During photosynthesis, plants …"
"""Tokyo is the capital city of J…"
"""The chemical symbol 'Fe' stand…"
…
"""The cheetah is the fastest lan…"
"""The Sahara Desert is located o…"
"""George Washington was the firs…"


In [5]:
next(iter(df["gold_answer"]))

'The mitochondria produce ATP and serve as the cell’s primary source of energy.'

In [6]:
scorer = rouge_scorer.RougeScorer(['rougeL'], use_stemmer=True)

ans = next(iter(df["gold_answer"]))
resp = next(iter(df["model_answer"]))

In [7]:
scorer.score_multi(["A B C", "D E"], "A C")

{'rougeL': Score(precision=1.0, recall=0.6666666666666666, fmeasure=0.8)}

In [16]:
scorer.score(ans, resp)['rougeL'].precision

0.5

In [9]:
df["gold_answer"].len()

20

In [12]:
for ans, resp in zip(df["gold_answer"], df["model_answer"]):
    print(scorer.score(ans, resp))

{'rougeL': Score(precision=0.5833333333333334, recall=0.5, fmeasure=0.5384615384615384)}
{'rougeL': Score(precision=1.0, recall=1.0, fmeasure=1.0)}
{'rougeL': Score(precision=0.4444444444444444, recall=0.6666666666666666, fmeasure=0.5333333333333333)}
{'rougeL': Score(precision=0.5714285714285714, recall=0.6666666666666666, fmeasure=0.6153846153846153)}
{'rougeL': Score(precision=0.5714285714285714, recall=0.5, fmeasure=0.5333333333333333)}
{'rougeL': Score(precision=0.6363636363636364, recall=0.875, fmeasure=0.7368421052631579)}
{'rougeL': Score(precision=1.0, recall=1.0, fmeasure=1.0)}
{'rougeL': Score(precision=0.75, recall=1.0, fmeasure=0.8571428571428571)}
{'rougeL': Score(precision=0.3, recall=0.5, fmeasure=0.37499999999999994)}
{'rougeL': Score(precision=1.0, recall=1.0, fmeasure=1.0)}
{'rougeL': Score(precision=1.0, recall=1.0, fmeasure=1.0)}
{'rougeL': Score(precision=0.42857142857142855, recall=0.3333333333333333, fmeasure=0.375)}
{'rougeL': Score(precision=0.5833333333333334

In [10]:
for i in range(df["gold_answer"].len()):
    ans = df["gold_answer"][i]
    resp = df["model_answer"][i]
    print(i, scorer.score(ans, resp))

0 {'rougeL': Score(precision=0.5833333333333334, recall=0.5, fmeasure=0.5384615384615384)}
1 {'rougeL': Score(precision=1.0, recall=1.0, fmeasure=1.0)}
2 {'rougeL': Score(precision=0.4444444444444444, recall=0.6666666666666666, fmeasure=0.5333333333333333)}
3 {'rougeL': Score(precision=0.5714285714285714, recall=0.6666666666666666, fmeasure=0.6153846153846153)}
4 {'rougeL': Score(precision=0.5714285714285714, recall=0.5, fmeasure=0.5333333333333333)}
5 {'rougeL': Score(precision=0.6363636363636364, recall=0.875, fmeasure=0.7368421052631579)}
6 {'rougeL': Score(precision=1.0, recall=1.0, fmeasure=1.0)}
7 {'rougeL': Score(precision=0.75, recall=1.0, fmeasure=0.8571428571428571)}
8 {'rougeL': Score(precision=0.3, recall=0.5, fmeasure=0.37499999999999994)}
9 {'rougeL': Score(precision=1.0, recall=1.0, fmeasure=1.0)}
10 {'rougeL': Score(precision=1.0, recall=1.0, fmeasure=1.0)}
11 {'rougeL': Score(precision=0.42857142857142855, recall=0.3333333333333333, fmeasure=0.375)}
12 {'rougeL': Score

In [11]:
df[17].select(pl.col("gold_answer", "model_answer")).rows()

[('George Washington was the first U.S. president.',
  'George Washington was the first president of the United States.')]

In [18]:
from nltk.translate import bleu_score

ans = next(iter(df["gold_answer"]))
resp = next(iter(df["model_answer"]))
bleu_score.sentence_bleu([ans], resp)

0.48217931778851963

In [ ]:
bleu_score.sentence_bleu([ans], resp)

0.48217931778851963

In [19]:
for i in range(df["gold_answer"].len()):
    ans = df["gold_answer"][i]
    resp = df["model_answer"][i]
    print(i, bleu_score.sentence_bleu([ans], resp))

0 0.48217931778851963
1 1.0
2 0.7115535295670167
3 0.6736725254446679
4 0.5181323157300696
5 0.577497597217841
6 1.0
7 0.8136410108781797
8 0.47886173800789306
9 1.0
10 1.0
11 0.7317978729128142
12 0.5441546696164566
13 0.5865102576625455
14 1.0
15 0.9379094111501866
16 0.6491555850394412
17 0.6482123272483757
18 0.45028900034315333
19 0.5868292902204161


In [15]:
ans = df["gold_answer"][15]
resp = df["model_answer"][15]

print(scorer.score(ans, resp))
print(bleu_score.sentence_bleu([ans], resp))

{'rougeL': Score(precision=0.5714285714285714, recall=0.5714285714285714, fmeasure=0.5714285714285714)}
0.9379094111501866


In [16]:
df[15].select(pl.col("gold_answer", "model_answer")).rows()

[('The fastest land animal is the cheetah.',
  'The cheetah is the fastest land animal.')]

In [22]:
from bert_score import score as score_bert

In [24]:
ans = next(iter(df["gold_answer"]))
resp = next(iter(df["model_answer"]))

score_bert([resp], [ans], lang='en', verbose=True, use_fast_tokenizer=True)

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


calculating scores...
computing bert embedding.


  0%|          | 0/1 [00:00<?, ?it/s]

computing greedy matching.


  0%|          | 0/1 [00:00<?, ?it/s]

done in 0.03 seconds, 37.58 sentences/sec


(tensor([0.9388]), tensor([0.9327]), tensor([0.9357]))

In [19]:
ans

'The mitochondria produce ATP and serve as the cell’s primary source of energy.'

In [20]:
resp

"Mitochondria are responsible for producing ATP, the cell's main energy currency."

In [21]:
testAns = "Abraham Lincoln was the 16th president of the United States"
testResp = "Abraham Lincoln was not the 16th president of the United States"

score_bert([testAns], [testResp], rescale_with_baseline=True, lang='en', verbose=True, use_fast_tokenizer=True)

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


calculating scores...
computing bert embedding.


  0%|          | 0/1 [00:00<?, ?it/s]

computing greedy matching.


  0%|          | 0/1 [00:00<?, ?it/s]

done in 0.02 seconds, 45.22 sentences/sec


(tensor([0.8887]), tensor([0.8060]), tensor([0.8473]))

In [22]:
testAns = "Abraham Lincoln was the 16th president of the United States"
testResp = "Lincoln was the 16th president of the US"

score_bert([testAns], [testResp], rescale_with_baseline=True, lang='en', verbose=True, use_fast_tokenizer=True)

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


calculating scores...
computing bert embedding.


  0%|          | 0/1 [00:00<?, ?it/s]

computing greedy matching.


  0%|          | 0/1 [00:00<?, ?it/s]

done in 0.02 seconds, 47.43 sentences/sec


(tensor([0.7910]), tensor([0.8656]), tensor([0.8283]))

In [23]:
testAns = "Abraham Lincoln was the 16th president of the United States"
testResp = "Lincoln was the 15th president of the US"

score_bert([testAns], [testResp], rescale_with_baseline=True, lang='en', verbose=True, use_fast_tokenizer=True)

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


calculating scores...
computing bert embedding.


  0%|          | 0/1 [00:00<?, ?it/s]

computing greedy matching.


  0%|          | 0/1 [00:00<?, ?it/s]

done in 0.02 seconds, 49.47 sentences/sec


(tensor([0.7687]), tensor([0.8267]), tensor([0.7979]))

In [24]:
score_bert([testAns, "Howdy"], [testResp, "Hello"], rescale_with_baseline=True, lang='en', verbose=True, use_fast_tokenizer=True)

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


calculating scores...
computing bert embedding.


  0%|          | 0/1 [00:00<?, ?it/s]

computing greedy matching.


  0%|          | 0/1 [00:00<?, ?it/s]

done in 0.03 seconds, 62.06 sentences/sec


(tensor([0.7687, 0.0674]), tensor([0.8267, 0.0707]), tensor([0.7979, 0.0705]))

In [25]:
df["model_answer"].to_list()

["Mitochondria are responsible for producing ATP, the cell's main energy currency.",
 'Albert Einstein proposed the theory of general relativity.',
 'During photosynthesis, plants absorb carbon dioxide from the air.',
 'Tokyo is the capital city of Japan.',
 "The chemical symbol 'Fe' stands for iron.",
 'The heart is responsible for pumping blood throughout the human body.',
 'Jupiter is the largest planet in the Solar System.',
 "William Shakespeare wrote the play 'Romeo and Juliet'.",
 'The freezing point of water in Celsius is 0 degrees.',
 'The Pacific Ocean is the largest ocean on Earth.',
 'Mars is known as the Red Planet.',
 'Leonardo da Vinci painted the Mona Lisa.',
 'DNA, or deoxyribonucleic acid, carries genetic information in nearly all living organisms.',
 'Paper was invented in ancient China during the Han Dynasty.',
 'The chemical formula for water is H2O.',
 'The cheetah is the fastest land animal.',
 'The Sahara Desert is located on the African continent.',
 'George Wa

In [26]:
score_bert(df["gold_answer"].to_list(), df["model_answer"].to_list(), rescale_with_baseline=True, lang='en', verbose=True, use_fast_tokenizer=True)

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


calculating scores...
computing bert embedding.


  0%|          | 0/1 [00:00<?, ?it/s]

computing greedy matching.


  0%|          | 0/1 [00:00<?, ?it/s]

done in 0.16 seconds, 122.61 sentences/sec


(tensor([0.6008, 1.0000, 0.8124, 0.7886, 0.6499, 0.8843, 1.0000, 0.9622, 0.6501,
         1.0000, 1.0000, 0.7281, 0.8850, 0.9572, 1.0000, 0.8587, 0.9351, 0.9156,
         0.8130, 0.8570]),
 tensor([0.6365, 1.0000, 0.6711, 0.7392, 0.6516, 0.7689, 1.0000, 0.8879, 0.4480,
         1.0000, 1.0000, 0.8138, 0.2658, 0.7732, 1.0000, 0.8850, 0.8602, 0.8138,
         0.6348, 0.6776]),
 tensor([0.6193, 1.0000, 0.7413, 0.7642, 0.6513, 0.8263, 1.0000, 0.9249, 0.5479,
         1.0000, 1.0000, 0.7710, 0.5587, 0.8640, 1.0000, 0.8720, 0.8975, 0.8644,
         0.7229, 0.7663]))